In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

fm._rebuild()
plt.rc('font', family='NanumGothic')

In [ ]:
# # 지수표현 제거
# pd.options.display.float_format = '{:.5f}'.format
# np.set_printoptions(precision=6, suppress=True)
# # ax.get_yaxis().get_major_formatter().set_scientific(False)

In [ ]:
df = pd.read_csv('./부동산_최종_데이터.csv')

In [ ]:
df.head()

,거래년도,자치구명,법정동명,일반주소,상세주소,건물명,건물주용도,건축년도,역세권여부,주변역,층정보,건물면적,규격,평수,평단가,위도,경도,거래가격
0,2010,강남구,대치동,강남구 대치동 동부센트레빌 아파트,강남구 대치동 0670-0000,동부센트레빌,아파트,2005,1,한티 수인분당선,24,161.47,대형,49,53877551,37.492796,127.055776,2640000000
1,2010,강남구,대치동,강남구 대치동 동부센트레빌 아파트,강남구 대치동 0670-0000,동부센트레빌,아파트,2005,1,한티 수인분당선,4,121.74,대형,37,48648649,37.492796,127.055776,1800000000
2,2010,강남구,대치동,강남구 대치동 동부센트레빌 아파트,강남구 대치동 0670-0000,동부센트레빌,아파트,2005,1,한티 수인분당선,25,145.83,대형,44,50454545,37.492796,127.055776,2220000000
3,2010,강남구,대치동,강남구 대치동 동부센트레빌 아파트,강남구 대치동 0670-0000,동부센트레빌,아파트,2005,1,한티 수인분당선,16,121.74,대형,37,51621622,37.492796,127.055776,1910000000
4,2010,강남구,대치동,강남구 대치동 동부센트레빌 아파트,강남구 대치동 0670-0000,동부센트레빌,아파트,2005,1,한티 수인분당선,19,161.47,대형,49,52755102,37.492796,127.055776,2585000000


In [ ]:
df = df.astype({ "거래년도" : object })

In [ ]:
df_SongPa = df[(df["자치구명"] == "송파구") & (df["거래년도"] > 2015)].reset_index( drop = True )
df_YongSan = df[(df["자치구명"] == "용산구") & (df["거래년도"] > 2015)].reset_index( drop = True )
df_GangNam = df[(df["자치구명"] == "강남구") & (df["거래년도"] > 2015)].reset_index( drop = True )
df_DoBong = df[(df["자치구명"] == "도봉구") & (df["거래년도"] > 2015)].reset_index( drop = True )
df_GuRo = df[(df["자치구명"] == "구로구") & (df["거래년도"] > 2015)].reset_index( drop = True )
df_YangCheon = df[(df["자치구명"] == "양천구") & (df["거래년도"] > 2015)].reset_index( drop = True )
df_MaPo = df[(df["자치구명"] == "마포구") & (df["거래년도"] > 2015)].reset_index( drop = True )
df_DongJak = df[(df["자치구명"] == "동작구") & (df["거래년도"] > 2015)].reset_index( drop = True )
df_DongDaeMoon = df[(df["자치구명"] == "동대문구") & (df["거래년도"] > 2015)].reset_index( drop = True )

In [ ]:
df_SongPa

,거래년도,자치구명,법정동명,건물면적,층정보,건물주용도,건축년도,거래가격,건물명,평수,규격,평단가격
0,2016,송파구,방이동,26.98,9,오피스텔,2004,150000000,보보스타워,8,소형,18750000
1,2016,송파구,잠실동,84.83,23,아파트,2007,1015000000,트리지움,26,중형,39038462
2,2016,송파구,잠실동,119.93,14,아파트,2008,1440000000,잠실엘스,36,대형,40000000
3,2016,송파구,잠실동,84.80,17,아파트,2008,1055000000,잠실엘스,26,중형,40576923
4,2016,송파구,잠실동,84.88,24,아파트,2008,1050000000,잠실엘스,26,중형,40384615
...,...,...,...,...,...,...,...,...,...,...,...,...
51928,2020,송파구,송파동,78.30,1,연립주택,2000,480000000,은성빌라,24,중형,20000000
51929,2020,송파구,석촌동,97.63,4,연립주택,2001,580000000,봄날햇살,30,대형,19333333
51930,2020,송파구,잠실동,84.99,11,아파트,2008,2100000000,리센츠,26,중형,80769231
51931,2020,송파구,잠실동,76.50,13,아파트,1978,2230000000,주공아파트 5단지,23,중형,96956522


In [ ]:
lst = [ df_SongPa, df_YongSan, df_GangNam, df_DoBong, df_GuRo, df_YangCheon]

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import time
import warnings
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GridSearchCV

In [ ]:
def DT( df, range ) :

  df_cat = df[['거래년도', '법정동명', '건물주용도']]
  X_cat = pd.get_dummies(df_cat)

  df_scaled = df[['층정보', '건축년도', '평수']]
  scaler = StandardScaler()

  x = scaler.fit_transform(df_scaled)
  X_scaled = pd.DataFrame(x, index=df_scaled.index, columns=df_scaled.columns)

  X_data = pd.concat([X_cat, X_scaled], axis=1)
  y_target = df['거래가격']

  X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.3, random_state=0)

  dt = DecisionTreeRegressor()

  parameters = {'max_depth': range}

  grid_dt = GridSearchCV(dt, param_grid=parameters, scoring = 'r2')

  grid_dt.fit(X_train, y_train)

  print('최적 하이퍼 파라미터:\n', grid_dt.best_params_)
  print('최고 예측 정확도: {0:.4f}'.format(grid_dt.best_score_))
  print( '' )

In [ ]:
def RF( df, range ) :

  df_cat = df[['거래년도', '법정동명', '건물주용도']]
  X_cat = pd.get_dummies(df_cat)

  df_scaled = df[['층정보', '건축년도', '평수']]
  scaler = StandardScaler()

  x = scaler.fit_transform(df_scaled)
  X_scaled = pd.DataFrame(x, index=df_scaled.index, columns=df_scaled.columns)

  X_data = pd.concat([X_cat, X_scaled], axis=1)
  y_target = df['거래가격']

  X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.3, random_state=0)

  rf = RandomForestRegressor()

  parameters = {'n_estimators': range}

  grid_rf = GridSearchCV(rf, param_grid=parameters, scoring = 'r2')

  grid_rf.fit(X_train, y_train)

  print('최적 하이퍼 파라미터:\n', grid_rf.best_params_)
  print('최고 예측 정확도: {0:.4f}'.format(grid_rf.best_score_))
  print( '' )

In [ ]:
def GB( df, range ) :

  df_cat = df[['거래년도', '법정동명', '건물주용도']]
  X_cat = pd.get_dummies(df_cat)

  df_scaled = df[['층정보', '건축년도', '평수']]
  scaler = StandardScaler()

  x = scaler.fit_transform(df_scaled)
  X_scaled = pd.DataFrame(x, index=df_scaled.index, columns=df_scaled.columns)

  X_data = pd.concat([X_cat, X_scaled], axis=1)
  y_target = df['거래가격']

  X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.3, random_state=0)

  gb = GradientBoostingRegressor()

  parameters = {'n_estimators': range}

  grid_gb = GridSearchCV(gb, param_grid=parameters, scoring = 'r2')

  grid_gb.fit(X_train, y_train)

  print('최적 하이퍼 파라미터:\n', grid_gb.best_params_)
  print('최고 예측 정확도: {0:.4f}'.format(grid_gb.best_score_))
  print( '' )

In [ ]:
def XGB( df, range ) :

  df_cat = df[['거래년도', '법정동명', '건물주용도']]
  X_cat = pd.get_dummies(df_cat)

  df_scaled = df[['층정보', '건축년도', '평수']]
  scaler = StandardScaler()

  x = scaler.fit_transform(df_scaled)
  X_scaled = pd.DataFrame(x, index=df_scaled.index,
                          columns=df_scaled.columns)

  X_data = pd.concat([X_cat, X_scaled], axis=1)
  y_target = df['거래가격']

  X_train, X_test, y_train, y_test = train_test_split(X_data,
                                                      y_target,
                                                      test_size=0.3,
                                                      random_state=0)

  xgb = XGBRegressor()

  parameters = {'n_estimators': range}

  grid_xgb = GridSearchCV(xgb, param_grid=parameters, scoring = 'r2')

  grid_xgb.fit(X_train, y_train)

  print('최고 예측 정확도: {0:.4f}'.format(grid_xgb.best_score_))
  print( '' )

In [ ]:
def LGBM( df, range ) :

  df_cat = df[['거래년도', '법정동명', '건물주용도']]
  X_cat = pd.get_dummies(df_cat)

  df_scaled = df[['층정보', '건축년도', '평수']]
  scaler = StandardScaler()

  x = scaler.fit_transform(df_scaled)
  X_scaled = pd.DataFrame(x, index=df_scaled.index, columns=df_scaled.columns)

  X_data = pd.concat([X_cat, X_scaled], axis=1)
  y_target = df['거래가격']

  X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.3, random_state=0)

  lgbm = LGBMRegressor()

  parameters = {'n_estimators': range}

  grid_lgbm = GridSearchCV(lgbm, param_grid=parameters, scoring = 'r2')

  grid_lgbm.fit(X_train, y_train)

  print('최적 하이퍼 파라미터:\n', grid_lgbm.best_params_)
  print('최고 예측 정확도: {0:.4f}'.format(grid_lgbm.best_score_))
  print( '' )

In [ ]:
for i in lst :
  RF( i, [1, 10, 50, 100, 200, 400, 700, 1000])

최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.9372

최적 하이퍼 파라미터:
 {'n_estimators': 100}
최고 예측 정확도: 0.8739

최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.9055

최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.8609

최적 하이퍼 파라미터:
 {'n_estimators': 400}
최고 예측 정확도: 0.8679

최적 하이퍼 파라미터:
 {'n_estimators': 700}
최고 예측 정확도: 0.9078



In [ ]:
for i in lst :
  GB( i, [1, 10, 50, 100, 200, 400, 700, 1000])

최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.9305

최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.8835

최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.8814

최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.8544

최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.8472

최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.9040



In [ ]:
for i in lst :
  DT( i, [1, 10, 50, 100, 200, 400, 700, 1000])

최적 하이퍼 파라미터:
 {'max_depth': 100}
최고 예측 정확도: 0.9121

최적 하이퍼 파라미터:
 {'max_depth': 100}
최고 예측 정확도: 0.8057

최적 하이퍼 파라미터:
 {'max_depth': 700}
최고 예측 정확도: 0.8557

최적 하이퍼 파라미터:
 {'max_depth': 10}
최고 예측 정확도: 0.8272

최적 하이퍼 파라미터:
 {'max_depth': 400}
최고 예측 정확도: 0.8180

최적 하이퍼 파라미터:
 {'max_depth': 10}
최고 예측 정확도: 0.8818



In [ ]:
for i in lst :
  XGB( i, [1, 10, 50, 100, 200, 400, 700, 1000])

[16:36:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:36:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:36:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:36:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:36:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:36:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:36:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:36:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [ ]:
for i in lst :
  LGBM( i, [1, 10, 50, 100, 200, 400, 700, 1000])

최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.9474

최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.8968

최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.9248

최적 하이퍼 파라미터:
 {'n_estimators': 700}
최고 예측 정확도: 0.8815

최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.8905

최적 하이퍼 파라미터:
 {'n_estimators': 700}
최고 예측 정확도: 0.9239



- 강남 dt 1차 : 최적 하이퍼 파라미터:
 {'max_depth': 70}
최고 예측 정확도: 0.9159
- 강남 dt 2차 : 최적 하이퍼 파라미터:
 {'max_depth': 60}
최고 예측 정확도: 0.9160
- 강남 dt 최종 : 최적 하이퍼 파라미터:
 {'max_depth': 55}
최고 예측 정확도: 0.9149
- 강남 xgb 최종 : 최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.9586
- 강남 lgb 최종 : 최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.9727

- 용산 dt 1차 : 최적 하이퍼 파라미터:
 {'max_depth': 50}
최고 예측 정확도: 0.9333
- 용산 dt 최종 : 최적 하이퍼 파라미터:
 {'max_depth': 45}
최고 예측 정확도: 0.9341
- 용산 xgb 최정 : 최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.9551

- 송파 dt 1차 : 최적 하이퍼 파라미터:
 {'max_depth': 50}
최고 예측 정확도: 0.9537
- 송파 dt 최종 : 최적 하이퍼 파라미터:
 {'max_depth': 50}
최고 예측 정확도: 0.9538

- 송파 xgb 1차 : 최적 하이퍼 파라미터:
 {'n_estimators': 200}
최고 예측 정확도: 0.9315
- 송파 xgb 2차 : 최적 하이퍼 파라미터:
 {'n_estimators': 400}
최고 예측 정확도: 0.9456
- 송파 xgb 최종 : 최적 하이퍼 파라미터:
 {'n_estimators': 1000}
최고 예측 정확도: 0.9586


In [ ]:
# 모델 셋 검증

dt = DecisionTreeRegressor()

# cross_val_score( )로 cv 셋으로 MSE 를 구한 뒤 이를 기반으로 다시  RMSE 구함.
neg_mse_scores = cross_val_score(dt, X_data, y_target, scoring="neg_mean_squared_error", cv = 40)
rmse_scores  = np.sqrt(-1 * neg_mse_scores)
avg_rmse = np.mean(rmse_scores)

# cross_val_score(scoring="neg_mean_squared_error")로 반환된 값은 모두 음수
# print(' cv수 의 개별 Negative MSE scores: ', np.round(neg_mse_scores, 2))
# print(' cv수 의 개별 RMSE scores : ', np.round(rmse_scores, 2))
print(' 40 folds 의 평균 RMSE : {0:.3f} '.format(avg_rmse))

 cv수 의 평균 RMSE : 198461817.472 
